In [57]:
import pandas as pd
import plotly
from pathlib import Path
import re
import ast
import pandas as pd
import numpy as np
from numpy.fft import fft , ifft , fftfreq



In [58]:
base = Path('./filtered_data/')

groups_dfs = {}

for csv_path in sorted(base.glob('group*/*.csv')):
    group = csv_path.parent.name
    m = re.search(r'dataset_user_(\d+)_train\.csv', csv_path.name)
    if not m:
        continue
    user_id = int(m.group(1))
    df = pd.read_csv(csv_path)
    groups_dfs.setdefault(group, {})[user_id] = df

groups_dfs["group0"][0]

,Unnamed: 0,day,label,hr_maxHeartRate,hr_minHeartRate,hr_restingHeartRate,hr_lastSevenDaysAvgRestingHeartRate,hr_time_series,resp_lowestRespirationValue,resp_highestRespirationValue,...,sleep_highestRespirationValue,sleep_awakeCount,sleep_avgSleepStress,sleep_avgHeartRate,len_hr_time_series,len_resp_time_series,len_stress_time_series,coverage_hr,coverage_resp,coverage_stress
0,0,0,79,114,39,40,49,"[55, 51, 46, 47, 45, 44, 50, 51, 46, 46, 49, 4...",7.0,20.0,...,18.0,0.0,6.0,47.0,702,720,480,0.998575,0.868056,0.822917
1,2,2,87,115,43,44,46,"[71, 75, 73, 72, 72, 67, 65, 68, 73, 71, 72, 8...",6.0,19.0,...,16.0,1.0,17.0,53.0,720,720,480,1.000000,0.862500,0.831250
2,3,3,54,109,42,45,46,"[59, 64, 64, 57, 59, 56, 56, 53, 51, 52, 47, 4...",7.0,20.0,...,16.0,2.0,14.0,52.0,598,720,480,1.000000,0.776389,0.758333
3,5,5,75,94,39,40,47,"[73, 67, 65, 59, 59, 64, 60, 53, 49, 49, 49, 4...",7.0,19.0,...,17.0,0.0,7.0,46.0,342,343,229,0.997076,0.976676,0.969432
4,6,6,89,108,46,49,45,"[71, 71, 72, 68, 68, 73, 74, 74, 71, 73, 70, 7...",7.0,22.0,...,18.0,1.0,23.0,57.0,720,720,480,1.000000,0.941667,0.908333
5,7,7,82,110,41,44,46,"[77, 67, 50, 45, 48, 50, 57, 53, 58, 60, 60, 5...",6.0,20.0,...,17.0,1.0,14.0,51.0,720,720,480,1.000000,0.950000,0.912500
6,8,8,85,110,40,43,46,"[51, 55, 50, 53, 55, 55, 52, 58, 58, 64, 68, 5...",7.0,21.0,...,17.0,0.0,13.0,48.0,720,720,480,1.000000,0.888889,0.829167
7,9,9,88,128,44,45,46,"[65, 63, 67, 65, 65, 60, 53, 58, 46, 46, 48, 4...",7.0,22.0,...,16.0,0.0,14.0,51.0,720,720,480,1.000000,0.834722,0.777083
8,10,10,85,115,39,44,46,"[65, 65, 63, 64, 68, 71, 68, 66, 66, 76, 64, 6...",8.0,20.0,...,18.0,1.0,12.0,50.0,720,720,480,1.000000,0.945833,0.902083
9,12,13,79,125,44,46,45,"[49, 47, 45, 46, 46, 47, 47, 47, 47, 48, 48, 5...",8.0,21.0,...,15.0,1.0,13.0,51.0,720,720,480,1.000000,0.856944,0.810417


CLEANING

In [59]:
def convert_timeseries_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    def try_parse_list(x):
        if isinstance(x, str) and x.strip().startswith('[') and x.strip().endswith(']'):
            try:
                return ast.literal_eval(x)
            except (ValueError, SyntaxError):
                return x
        return x

    for col in df.columns:
        df[col] = df[col].apply(try_parse_list)

    return df

def clean_ts(ts):
    if not isinstance(ts, list):
        return [np.nan]
    return [np.nan if (x is None or x < 0) else x for x in ts]

for group, users in groups_dfs.items():
    for user_id, df in users.items():
        groups_dfs[group][user_id] = convert_timeseries_columns(df)

for group, users in groups_dfs.items():
    for user_id, df in users.items():
        for col in df.columns:
            if 'time_series' in col:
                df[col] = df[col].apply(clean_ts)
        df['who'] = user_id
        groups_dfs[group][user_id] = df

groups_dfs["group0"][0]

,Unnamed: 0,day,label,hr_maxHeartRate,hr_minHeartRate,hr_restingHeartRate,hr_lastSevenDaysAvgRestingHeartRate,hr_time_series,resp_lowestRespirationValue,resp_highestRespirationValue,...,sleep_awakeCount,sleep_avgSleepStress,sleep_avgHeartRate,len_hr_time_series,len_resp_time_series,len_stress_time_series,coverage_hr,coverage_resp,coverage_stress,who
0,0,0,79,114,39,40,49,"[55, 51, 46, 47, 45, 44, 50, 51, 46, 46, 49, 4...",7.0,20.0,...,0.0,6.0,47.0,[nan],[nan],[nan],0.998575,0.868056,0.822917,0
1,2,2,87,115,43,44,46,"[71, 75, 73, 72, 72, 67, 65, 68, 73, 71, 72, 8...",6.0,19.0,...,1.0,17.0,53.0,[nan],[nan],[nan],1.000000,0.862500,0.831250,0
2,3,3,54,109,42,45,46,"[59, 64, 64, 57, 59, 56, 56, 53, 51, 52, 47, 4...",7.0,20.0,...,2.0,14.0,52.0,[nan],[nan],[nan],1.000000,0.776389,0.758333,0
3,5,5,75,94,39,40,47,"[73, 67, 65, 59, 59, 64, 60, 53, 49, 49, 49, 4...",7.0,19.0,...,0.0,7.0,46.0,[nan],[nan],[nan],0.997076,0.976676,0.969432,0
4,6,6,89,108,46,49,45,"[71, 71, 72, 68, 68, 73, 74, 74, 71, 73, 70, 7...",7.0,22.0,...,1.0,23.0,57.0,[nan],[nan],[nan],1.000000,0.941667,0.908333,0
5,7,7,82,110,41,44,46,"[77, 67, 50, 45, 48, 50, 57, 53, 58, 60, 60, 5...",6.0,20.0,...,1.0,14.0,51.0,[nan],[nan],[nan],1.000000,0.950000,0.912500,0
6,8,8,85,110,40,43,46,"[51, 55, 50, 53, 55, 55, 52, 58, 58, 64, 68, 5...",7.0,21.0,...,0.0,13.0,48.0,[nan],[nan],[nan],1.000000,0.888889,0.829167,0
7,9,9,88,128,44,45,46,"[65, 63, 67, 65, 65, 60, 53, 58, 46, 46, 48, 4...",7.0,22.0,...,0.0,14.0,51.0,[nan],[nan],[nan],1.000000,0.834722,0.777083,0
8,10,10,85,115,39,44,46,"[65, 65, 63, 64, 68, 71, 68, 66, 66, 76, 64, 6...",8.0,20.0,...,1.0,12.0,50.0,[nan],[nan],[nan],1.000000,0.945833,0.902083,0
9,12,13,79,125,44,46,45,"[49, 47, 45, 46, 46, 47, 47, 47, 47, 48, 48, 5...",8.0,21.0,...,1.0,13.0,51.0,[nan],[nan],[nan],1.000000,0.856944,0.810417,0


In [60]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Remove TensorFlow/Keras imports
# from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


def scale_data(df):
    # drop rows with all NaN
    df = df.dropna(how='all')
    # fill remaining NaNs per column with forward fill then back fill
    df = df.fillna(method='ffill').fillna(method='bfill')
    # still may have NaNs at boundaries; replace with column means
    df = df.apply(lambda col: col.fillna(col.mean()))

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    return scaled, scaler


def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        x_win = X[i:i + seq_length]
        y_val = y[i + seq_length]
        if np.isnan(x_win).any() or np.isnan(y_val):
            continue
        X_seq.append(x_win)
        y_seq.append(y_val)
    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32)


# PyTorch LSTM model definition
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size1=128, dropout=0.2):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size1, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size1, 1)

    def forward(self, x):
        # x: [batch, seq_len, input_size]
        out1, _ = self.lstm1(x)
        out1 = self.dropout1(out1)
        last = out1[:, -1, :]
        return self.fc(last)

# Utility for MAE
def mae(pred, target):
    return torch.mean(torch.abs(pred - target))

In [61]:
merged_group = {}

# 1. scegli la sorgente dati
source_dict = groups_dfs


for group, users in source_dict.items():
    try:
        merged_group[group] = pd.concat(users.values(), ignore_index=True)
    except ValueError:
        merged_group[group] = pd.DataFrame()

x_all, y_all = [], []


# 3. selezione feature numeriche sicure
def numeric_features(df):
    num_df = (
        df
        .loc[:, ~df.columns.str.contains('time_series')]
        .select_dtypes(include=np.number)
    )
    # assicurati che tutte le righe siano numeriche o NaN
    return [
        c for c in num_df.columns
        if num_df[c].map(lambda v: isinstance(v, (int, float, np.number)) or pd.isna(v)).all()
    ]


# 4. costruzione X, y
for df in merged_group.values():

    if df.empty:
        continue

    features = [c for c in numeric_features(df) if c != 'label']
    if not features:
        continue

    X, _ = scale_data(df[features])
    y = df['label'].astype(np.float32).values

    x_all.append(X)
    y_all.append(y)


/tmp/ipykernel_5661/1807454652.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1807454652.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1807454652.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1807454652.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1807454652.py:17: FutureWarning: DataFrame.fillna with 'meth

# Data audit and preprocessing improvements

We will:
- Verify feature/label columns per group and fix mismatches.
- Clip outliers using quantiles to reduce noise.
- Add simple rolling statistics features (mean/std) to improve signal.
- Rebuild `x_all` and `y_all` consistently for downstream training.
- Report per-group basic stats and missing values.

In [ ]:
# Audit data, engineer rolling features, clip outliers, and rebuild x_all/y_all
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


data_src = merged_group

# Rolling features for hr/resp if present
def add_rolling_feats(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for base in ['hr', 'resp']:
        if base in df.columns:
            s = df[base].astype(np.float32)
            df[f'{base}_roll_mean_5'] = s.rolling(window=5, min_periods=1).mean()
            df[f'{base}_roll_std_5'] = s.rolling(window=5, min_periods=1).std().fillna(0.0)
            df[f'{base}_roll_mean_10'] = s.rolling(window=10, min_periods=1).mean()
            df[f'{base}_roll_std_10'] = s.rolling(window=10, min_periods=1).std().fillna(0.0)
    return df

# Clip outliers on numeric columns by quantiles
def clip_outliers(df: pd.DataFrame, q_low=0.01, q_high=0.99) -> pd.DataFrame:
    df = df.copy()
    num_cols = df.select_dtypes(include=[np.number]).columns
    for c in num_cols:
        ql = df[c].quantile(q_low)
        qh = df[c].quantile(q_high)
        df[c] = df[c].clip(ql, qh)
    return df

# Clean NaNs (forward/back fill, then mean)
def clean_nans(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.dropna(how='all')
    df = df.fillna(method='ffill').fillna(method='bfill')
    df = df.apply(lambda col: col.fillna(col.mean()) if col.isna().any() else col)
    return df

enhanced_groups = {}
report = {}

for g, df in data_src.items():
    # Start from numeric scalar features only

    # Add rolling features (only if base columns exist)
    #drop column Unnamed: 0
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    df = df.drop(columns=['day'], errors='ignore')
    df = df.drop(columns=['act_distance'], errors='ignore')

    

    df = add_rolling_feats(df)
    # Recompute numeric feature columns including new ones
    feature_cols = [c for c in numeric_features(df) if c != 'label']


    df[feature_cols] = clip_outliers(df[feature_cols])
    # Clean NaNs
    df = clean_nans(df)

    # Save enhanced
    enhanced_groups[g] = df
    # Basic report
    report[g] = {
        'rows': int(len(df)),
        'features': len(feature_cols),
        'missing_total': int(df.isna().sum().sum()),
        'label_min': float(df['label'].min()),
        'label_max': float(df['label'].max()),
        'label_mean': float(df['label'].mean())
    }

print('Data audit report per group:')
for g, r in report.items():
    print(g, r)

# Build scaled arrays for training
x_all = []
y_all = []
y_scalers = []

for g, df in enhanced_groups.items():
    feature_cols = [c for c in numeric_features(df) if c != 'label']
    if len(feature_cols) == 0:
        continue
    X = MinMaxScaler().fit_transform(df[feature_cols]).astype(np.float32)
    ys = MinMaxScaler()
    y = ys.fit_transform(df['label'].values.astype(np.float32).reshape(-1,1)).squeeze().astype(np.float32)
    x_all.append(X)
    y_all.append(y)
    y_scalers.append(ys)
    
# Update n_features based on enhanced feature count if available
if len(x_all) > 0:
    n_features = x_all[0].shape[1]
    print('n_features (enhanced):', n_features)
else:
    print('No numeric features available after enhancement; x_all is empty.')

/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'meth

Index(['label', 'hr_maxHeartRate', 'hr_minHeartRate', 'hr_restingHeartRate',
       'hr_lastSevenDaysAvgRestingHeartRate', 'hr_time_series',
       'resp_lowestRespirationValue', 'resp_highestRespirationValue',
       'resp_avgWakingRespirationValue', 'resp_avgSleepRespirationValue',
       'resp_avgTomorrowSleepRespirationValue', 'resp_time_series',
       'str_maxStressLevel', 'str_avgStressLevel', 'stress_time_series',
       'sleep_sleepTimeSeconds', 'sleep_napTimeSeconds',
       'sleep_unmeasurableSleepSeconds', 'sleep_deepSleepSeconds',
       'sleep_lightSleepSeconds', 'sleep_remSleepSeconds',
       'sleep_awakeSleepSeconds', 'act_totalCalories',
       'act_activeKilocalories', 'sleep_averageRespirationValue',
       'sleep_lowestRespirationValue', 'sleep_highestRespirationValue',
       'sleep_awakeCount', 'sleep_avgSleepStress', 'sleep_avgHeartRate',
       'len_hr_time_series', 'len_resp_time_series', 'len_stress_time_series',
       'coverage_hr', 'coverage_resp', 'covera

/tmp/ipykernel_5661/2624454098.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


# Per-user training and group ensemble (with optimized preprocessing and MAE in original units)

We will train one model per user within each group, then build a group-level ensemble by averaging the user models. Validation MAE is computed in original units via inverse scaling per user. Finally, we produce a compact MAE report per group and overall.

In [70]:
# Train per-user models on raw CSV time_series (minimal processing), ensemble per group, and final MAE report
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



def clean_nans(df):
    df = df.copy()
    df = df.dropna(how='all')
    df = df.fillna(method='ffill').fillna(method='bfill')
    df = df.apply(lambda col: col.fillna(col.mean()) if col.isna().any() else col)
    return df

# Build per-step raw features: align hr/resp/stress lengths to min length, no filtering/rolling/clipping
# Preserve ALL non-time_series columns exactly as in the CSVs.
def build_raw_steps(df):
    df = clean_nans(df)
    per_step = pd.DataFrame()
    # Identify non-time-series columns to copy verbatim
    non_ts_cols = [c for c in df.columns if 'time_series' not in c and c != 'label']
    for i in range(len(df)):
        row = df.iloc[i]
        # extract core time series
        ts_hr = np.array(row['hr_time_series'], dtype=np.float32) if ('hr_time_series' in row and isinstance(row['hr_time_series'], list)) else np.array([], dtype=np.float32)
        ts_resp = np.array(row['resp_time_series'], dtype=np.float32) if ('resp_time_series' in row and isinstance(row['resp_time_series'], list)) else np.array([], dtype=np.float32)
        ts_stress = np.array(row['stress_time_series'], dtype=np.float32) if ('stress_time_series' in row and isinstance(row['stress_time_series'], list)) else np.array([], dtype=np.float32)
        n_common = min(len(ts_hr), len(ts_resp), len(ts_stress))
        if n_common == 0:
            continue
        new_df = pd.DataFrame({'hr': ts_hr[:n_common], 'resp': ts_resp[:n_common], 'stress': ts_stress[:n_common]})
        # copy all non-time_series columns exactly as in CSV
        for c in non_ts_cols:
            new_df[c] = row[c]
        # label per row (broadcast)
        new_df['label'] = row['label']
        per_step = pd.concat([per_step, new_df], ignore_index=True)
    return per_step

# Assemble per-user data from raw CSVs
base_src = {}
for group, users in groups_dfs.items():
    base_src[group] = {}
    for uid, df in users.items():
        base_src[group][uid] = build_raw_steps(df)

# Scale features and y per user (retain original-unit reporting via inverse scaling)
per_user_data = {}
for group, users in base_src.items():
    per_user_data[group] = {}
    for uid, df in users.items():
        if 'label' not in df.columns:
            continue
        # features: ALL columns except 'label' (includes any newly added CSV columns)
        feature_cols = [c for c in df.columns if c != 'label']
        X_scaled = MinMaxScaler().fit_transform(df[feature_cols]).astype(np.float32)
        ys = MinMaxScaler()
        y_scaled = ys.fit_transform(df['label'].values.astype(np.float32).reshape(-1,1)).squeeze().astype(np.float32)
        per_user_data[group][uid] = (X_scaled, y_scaled, ys)

class AveragedModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)
    def forward(self, x):
        preds = [m(x) for m in self.models]
        return torch.mean(torch.stack(preds, dim=0), dim=0)

def train_user_model(X_scaled, y_scaled, seq_len):
    X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_len)
    X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_len)
    if len(X_train_seq) == 0 or len(X_val_seq) == 0:
        return None, None
    input_dim = X_train_seq.shape[-1]
    model = LSTMRegressor(input_size=input_dim).to(device)
    criterion = nn.SmoothL1Loss(beta=0.5)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_train_seq, dtype=torch.float32).to(device),
                                           torch.tensor(y_train_seq.reshape(-1,1), dtype=torch.float32).to(device)),
                             batch_size=256, shuffle=True)
    val_loader = DataLoader(TensorDataset(torch.tensor(X_val_seq, dtype=torch.float32).to(device),
                                         torch.tensor(y_val_seq.reshape(-1,1), dtype=torch.float32).to(device)),
                           batch_size=256, shuffle=False)
    best_mae, best_state, wait = float('inf'), None, 0
    for epoch in range(20):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad(set_to_none=True)
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            mae_sum, n = 0.0, 0
            for xb, yb in val_loader:
                preds = model(xb)
                mae_sum += torch.sum(torch.abs(preds - yb)).item()
                n += xb.size(0)
            val_mae = mae_sum / max(n,1)
        scheduler.step(val_mae)
        if val_mae < best_mae - 1e-6:
            best_mae, best_state, wait = val_mae, {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}, 0
        else:
            wait += 1
            if wait >= 5:
                break
    if best_state is not None:
        model.load_state_dict(best_state)
    return model, (X_val_seq, y_val_seq)

seq_len = 10
group_ensembles = {}
group_reports = {}
for group, users in per_user_data.items():
    user_models = []
    user_scalers = []
    for uid, (X_u, y_u, ys_u) in users.items():
        model_u, val_pack = train_user_model(X_u, y_u, seq_len)
        if model_u is not None:
            user_models.append(model_u.to(device).eval())
            user_scalers.append(ys_u)
    if len(user_models) == 0:
        continue
    group_ensembles[group] = AveragedModel(user_models).to(device).eval()
    mae_sum, n = 0.0, 0
    for uid, (X_u, y_u, ys_u) in users.items():
        X_train, X_val, y_train, y_val = train_test_split(X_u, y_u, test_size=0.2, shuffle=False)
        X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_len)
        if len(X_val_seq) == 0:
            continue
        X_val_t = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        with torch.no_grad():
            y_pred_scaled = group_ensembles[group](X_val_t).detach().cpu().numpy().reshape(-1)
        y_true_orig = ys_u.inverse_transform(y_val_seq.reshape(-1,1)).reshape(-1)
        y_pred_orig = ys_u.inverse_transform(y_pred_scaled.reshape(-1,1)).reshape(-1)
        mae_sum += float(np.sum(np.abs(y_pred_orig - y_true_orig)))
        n += y_true_orig.shape[0]
    group_mae = mae_sum / max(n,1)
    group_reports[group] = {"users": len(user_models), "val_MAE_orig": group_mae}

print("Final MAE report per group (original units):")
for g, rep in group_reports.items():
    print(g, rep)
if len(group_reports) > 0:
    overall_mae = float(np.mean([rep["val_MAE_orig"] for rep in group_reports.values()]))
    print({"overall_group_val_MAE_orig": overall_mae})

/tmp/ipykernel_5661/3059144346.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/3059144346.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/3059144346.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/3059144346.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/3059144346.py:17: FutureWarning: DataFrame.fillna with 'meth

Final MAE report per group (original units):
group0 {'users': 4, 'val_MAE_orig': 12.100616477017434}
group1 {'users': 5, 'val_MAE_orig': 10.11194801418747}
group2 {'users': 4, 'val_MAE_orig': 15.01833536538733}
group3 {'users': 5, 'val_MAE_orig': 10.253808163931335}
group4 {'users': 5, 'val_MAE_orig': 7.707548278362699}
group5 {'users': 5, 'val_MAE_orig': 11.874289276591536}
group6 {'users': 5, 'val_MAE_orig': 9.046882914104225}
group7 {'users': 5, 'val_MAE_orig': 13.648088689097563}
group8 {'users': 5, 'val_MAE_orig': 8.572101751172523}
{'overall_group_val_MAE_orig': 10.925957658872456}


In [ ]:
# Audit original per-user DataFrames and improve input data
import pandas as pd
import numpy as np

# Use original cleaned time-series source
if 'groups_dfs' not in globals() or not isinstance(groups_dfs, dict):
    raise NameError('groups_dfs not available; run the earlier loading/cleaning cells.')

def series_quality_stats(df):
    stats = {}
    for c in ['hr_time_series', 'resp_time_series', 'stress_time_series']:
        if c in df.columns:
            lengths = df[c].apply(lambda s: len(s) if isinstance(s, list) else 0)
            stats[c] = {
                'count': int((lengths>0).sum()),
                'min_len': int(lengths[lengths>0].min()) if (lengths>0).any() else 0,
                'median_len': int(lengths[lengths>0].median()) if (lengths>0).any() else 0,
                'max_len': int(lengths.max())
            }
    return stats

def resample_series(lst, target_len):
    if not isinstance(lst, list) or len(lst)==0: return []
    arr = np.asarray(lst, dtype=np.float32)
    x_old = np.linspace(0, 1, len(arr))
    x_new = np.linspace(0, 1, target_len)
    return np.interp(x_new, x_old, arr).astype(np.float32).tolist()

def improve_df(df, target_len=128):
    # Ensure lists and resample to common length for consistent features
    df = df.copy()
    for c in ['hr_time_series','resp_time_series','stress_time_series']:
        if c in df.columns:
            df[c] = df[c].apply(lambda s: resample_series(s, target_len))
    # Derive per-step arrays
    def to_array(col):
        return np.stack([np.asarray(x, dtype=np.float32) for x in col], axis=0)
    feats = {}
    if 'hr_time_series' in df.columns:
        hr = to_array(df['hr_time_series'])
        # per-user z-score normalize per row
        hr_mean = hr.mean(axis=1, keepdims=True)
        hr_std = hr.std(axis=1, keepdims=True) + 1e-6
        hr_z = (hr - hr_mean) / hr_std
        # rolling stats along time dim (window 5)
        w = 5
        # simple moving average via convolution
        kernel = np.ones((w,), dtype=np.float32)/w
        def mov_mean(a):
            return np.apply_along_axis(lambda r: np.convolve(r, kernel, mode='same'), 1, a)
        hr_ma5 = mov_mean(hr_z)
        # first difference (gradient-like)
        hr_diff1 = np.diff(hr_z, axis=1, prepend=hr_z[:, :1])
        feats['hr_mean'] = hr_mean.squeeze()
        feats['hr_std'] = hr_std.squeeze()
        feats['hr_ma5_mean'] = hr_ma5.mean(axis=1)
        feats['hr_ma5_std'] = hr_ma5.std(axis=1)
        feats['hr_diff1_mean'] = hr_diff1.mean(axis=1)
        feats['hr_diff1_std'] = hr_diff1.std(axis=1)
    if 'resp_time_series' in df.columns:
        resp = to_array(df['resp_time_series'])
        resp_mean = resp.mean(axis=1, keepdims=True)
        resp_std = resp.std(axis=1, keepdims=True) + 1e-6
        resp_z = (resp - resp_mean) / resp_std
        w = 5
        kernel = np.ones((w,), dtype=np.float32)/w
        def mov_mean(a):
            return np.apply_along_axis(lambda r: np.convolve(r, kernel, mode='same'), 1, a)
        resp_ma5 = mov_mean(resp_z)
        resp_diff1 = np.diff(resp_z, axis=1, prepend=resp_z[:, :1])
        feats['resp_mean'] = resp_mean.squeeze()
        feats['resp_std'] = resp_std.squeeze()
        feats['resp_ma5_mean'] = resp_ma5.mean(axis=1)
        feats['resp_ma5_std'] = resp_ma5.std(axis=1)
        feats['resp_diff1_mean'] = resp_diff1.mean(axis=1)
        feats['resp_diff1_std'] = resp_diff1.std(axis=1)
    # label mapping
    if 'stress_time_series' in df.columns and 'label' not in df.columns:
        # Use the last value as proxy label if consistent; else keep existing label
        last_vals = np.array([ (s[-1] if isinstance(s, list) and len(s)>0 else np.nan) for s in df['stress_time_series'] ], dtype=np.float32)
        df['label'] = last_vals
    # Construct improved feature frame
    feat_df = pd.DataFrame(feats) if len(feats)>0 else pd.DataFrame(index=range(len(df)))
    # include static cols if present (age/sex/etc.)
    static_cols = [c for c in df.columns if ('time_series' not in c and c != 'label')]
    for c in static_cols:
        feat_df[c] = df[c]
    # attach label
    if 'label' in df.columns:
        feat_df['label'] = df['label'].astype(np.float32)
    # clean NaNs
    feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
    feat_df = feat_df.apply(lambda col: col.fillna(col.mean()) if col.isna().any() else col)
    return feat_df

# Audit and build improved per-user feature DataFrames
audit = {}
improved_src = {}
for group, users in groups_dfs.items():
    improved_src[group] = {}
    for user_id, df in users.items():
        st = series_quality_stats(df)
        audit[(group, user_id)] = st
        improved_src[group][user_id] = improve_df(df, target_len=128)

print('Original per-user series quality (sample):')
for k, v in list(audit.items())[:5]:
    print(k, v)
print('Built improved feature frames per user for all groups.')

# Optionally: replace base_src used by per-user training with improved_src
rebuilded_dfs = improved_src
print('Updated rebuilded_dfs to improved per-user features (label included).')

/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: Fu

Original per-user series quality (sample):
('group0', 0) {'hr_time_series': {'count': 19, 'min_len': 342, 'median_len': 720, 'max_len': 720}, 'resp_time_series': {'count': 19, 'min_len': 343, 'median_len': 720, 'max_len': 720}, 'stress_time_series': {'count': 19, 'min_len': 229, 'median_len': 480, 'max_len': 480}}
('group0', 27) {'hr_time_series': {'count': 2, 'min_len': 622, 'median_len': 647, 'max_len': 672}, 'resp_time_series': {'count': 2, 'min_len': 720, 'median_len': 720, 'max_len': 720}, 'stress_time_series': {'count': 2, 'min_len': 480, 'median_len': 480, 'max_len': 480}}
('group0', 36) {'hr_time_series': {'count': 3, 'min_len': 604, 'median_len': 633, 'max_len': 691}, 'resp_time_series': {'count': 3, 'min_len': 719, 'median_len': 720, 'max_len': 720}, 'stress_time_series': {'count': 3, 'min_len': 480, 'median_len': 480, 'max_len': 480}}
('group0', 9) {'hr_time_series': {'count': 17, 'min_len': 637, 'median_len': 720, 'max_len': 720}, 'resp_time_series': {'count': 17, 'min_len'

/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat_df = feat_df.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_5661/1473908573.py:92: Fu